In [1]:
"""
Complete CAPTCHA Recognition Script for Kaggle
This script combines all components: dataset, model, training, and evaluation
"""

# ============================================================================
# IMPORTS
# ============================================================================
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn import preprocessing
from torch.utils.data import Dataset
from PIL import Image, ImageFile
import albumentations as A
from tqdm import tqdm


In [2]:

ImageFile.LOAD_TRUNCATED_IMAGES = True

# ============================================================================
# CONFIGURATION
# ============================================================================


class Config:
    # Paths - MODIFY THESE FOR YOUR KAGGLE DATASET
    TRAIN_DIR = "/kaggle/input/captcha-dataset-98k/Dataset/train"
    VAL_DIR = "/kaggle/input/captcha-dataset-98k/Dataset/val"

    # Image dimensions
    IMAGE_HEIGHT = 40
    IMAGE_WIDTH = 150

    # Training hyperparameters
    BATCH_SIZE = 32
    NUM_WORKERS = 2  # Kaggle typically uses 2
    EPOCHS = 150
    LEARNING_RATE = 1e-3

    # Device
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:

# ============================================================================
# DATASET CLASS
# ============================================================================


class ClassificationDataset(Dataset):
    """Dataset class for CAPTCHA images"""

    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        target = self.labels[idx]

        if self.transform is None:
            self.transform = A.Compose([
                A.Resize(Config.IMAGE_HEIGHT, Config.IMAGE_WIDTH),
                A.Normalize(mean=(0.485, 0.456, 0.406),
                            std=(0.229, 0.224, 0.225)),
            ])

        image = np.array(image)
        transformed = self.transform(image=image)
        image = transformed["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)

        return {
            "images": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(target, dtype=torch.long)
        }


In [4]:

# ============================================================================
# MODEL ARCHITECTURE
# ============================================================================


class CaptchaModel(nn.Module):
    """CNN + GRU model with CTC Loss for CAPTCHA recognition"""

    def __init__(self, num_characters):
        super(CaptchaModel, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=128, kernel_size=3, padding=1)
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=128, out_channels=64, kernel_size=3, padding=1)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Linear layer
        self.linear = nn.Linear(64*10, 64)  # 64 channels * height 10
        self.dropout = nn.Dropout(0.3)

        # GRU layers
        self.gru = nn.GRU(64, 32, bidirectional=True, num_layers=2,
                          batch_first=True, dropout=0.3)

        # Classifier
        self.classifier = nn.Linear(32*2, num_characters+1)  # +1 for CTC blank

    def forward(self, images, targets=None):
        batch_size, channels, height, width = images.size()

        # Conv block 1
        x = F.relu(self.conv1(images))
        x = self.max_pool1(x)

        # Conv block 2
        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)

        # Reshape for RNN: batch, width, channels*height
        x = x.permute(0, 3, 1, 2)
        x = x.view(batch_size, x.size(1), -1)

        # Linear + Dropout
        x = self.linear(x)
        x = self.dropout(x)

        # GRU
        x, _ = self.gru(x)

        # Classification
        x = self.classifier(x)

        # Permute for CTC Loss: width, batch, num_characters+1
        x = x.permute(1, 0, 2)

        # Calculate loss if targets provided
        if targets is not None:
            log_softmax = F.log_softmax(x, dim=2)
            input_lengths = torch.full(
                size=(batch_size,), fill_value=x.size(0), dtype=torch.long)
            target_lengths = torch.full(
                size=(batch_size,), fill_value=targets.size(1), dtype=torch.long)

            ctc_loss = nn.CTCLoss(blank=0)
            loss = ctc_loss(log_softmax, targets,
                            input_lengths, target_lengths)
            return x, loss

        return x, None

# ============================================================================
# TRAINING AND EVALUATION FUNCTIONS
# ============================================================================



In [5]:

def train_fn(model, data_loader, optimizer, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0

    for data in tqdm(data_loader, desc="Training", total=len(data_loader)):
        # Move data to device
        for key, value in data.items():
            data[key] = value.to(device)

        # Forward pass
        _, loss = model(**data)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(data_loader)


def eval_fn(model, data_loader, device):
    """Evaluate the model"""
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for data in tqdm(data_loader, desc="Validation", total=len(data_loader)):
            # Move data to device
            for key, value in data.items():
                data[key] = value.to(device)

            # Forward pass
            _, loss = model(**data)
            total_loss += loss.item()

    return total_loss / len(data_loader)


def decode_predictions(preds, lbl_enc):
    """Decode model predictions to text"""
    decoded = []
    for pred in preds:
        # Remove duplicates and blanks (CTC decoding)
        pred_chars = []
        prev_char = None
        for p in pred:
            if p != 0 and p != prev_char:  # 0 is blank
                pred_chars.append(p - 1)  # Remove the +1 offset
            prev_char = p

        # Convert to characters
        if len(pred_chars) > 0:
            text = ''.join(lbl_enc.inverse_transform(pred_chars))
        else:
            text = ''
        decoded.append(text)

    return decoded


In [6]:

# ============================================================================
# MAIN TRAINING FUNCTION
# ============================================================================


def run_training():
    """Main training pipeline"""

    print("="*70)
    print("CAPTCHA RECOGNITION TRAINING")
    print("="*70)

    # ========== STEP 1: LOAD DATA ==========
    print("\n[1/6] Loading image paths...")
    train_images = glob.glob(os.path.join(Config.TRAIN_DIR, '*.jpg'))
    train_targets_orig = [x.split(os.sep)[-1].split('.')[0]
                          for x in train_images]

    test_images = glob.glob(os.path.join(Config.VAL_DIR, '*.jpg'))
    test_targets_orig = [x.split(os.sep)[-1].split('.')[0]
                         for x in test_images]

    print(f"   Train images: {len(train_images)}")
    print(f"   Val images: {len(test_images)}")

    # ========== STEP 2: ENCODE LABELS ==========
    print("\n[2/6] Encoding labels...")

    # Split each captcha into characters
    train_targets = [[c for c in x] for x in train_targets_orig]
    test_targets = [[c for c in x] for x in test_targets_orig]

    # Flatten all characters
    targets_flat = [c for clist in train_targets for c in clist]
    targets_flat += [c for clist in test_targets for c in clist]

    # Fit LabelEncoder on all unique characters
    lbl_enc = preprocessing.LabelEncoder()
    lbl_enc.fit(targets_flat)

    print(f"   Unique characters: {len(lbl_enc.classes_)}")
    print(f"   Characters: {sorted(lbl_enc.classes_)}")

    # Encode targets
    train_targets_enc = [lbl_enc.transform(x) for x in train_targets]
    train_targets_enc = np.array(train_targets_enc) + 1  # Reserve 0 for blank

    test_targets_enc = [lbl_enc.transform(x) for x in test_targets]
    test_targets_enc = np.array(test_targets_enc) + 1

    # ========== STEP 3: CREATE DATASETS ==========
    print("\n[3/6] Creating datasets and dataloaders...")

    train_dataset = ClassificationDataset(
        image_paths=train_images,
        labels=train_targets_enc
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=Config.BATCH_SIZE,
        shuffle=True,
        num_workers=Config.NUM_WORKERS
    )

    test_dataset = ClassificationDataset(
        image_paths=test_images,
        labels=test_targets_enc
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=Config.BATCH_SIZE,
        shuffle=False,
        num_workers=Config.NUM_WORKERS
    )

    print(f"   Train batches: {len(train_loader)}")
    print(f"   Val batches: {len(test_loader)}")

    # ========== STEP 4: INITIALIZE MODEL ==========
    print("\n[4/6] Initializing model...")

    model = CaptchaModel(num_characters=len(lbl_enc.classes_))
    model.to(Config.DEVICE)

    print(
        f"   Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"   Device: {Config.DEVICE}")

    # Optimizer and scheduler
    optimizer = torch.optim.Adam(model.parameters(), lr=Config.LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, factor=0.5, patience=5
    )

    # ========== STEP 5: TRAINING LOOP ==========
    print("\n[5/6] Starting training...")
    print("="*70)

    best_val_loss = float('inf')

    for epoch in range(Config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{Config.EPOCHS}")
        print("-" * 70)

        # Train
        train_loss = train_fn(model, train_loader, optimizer, Config.DEVICE)
        print(f"   Train Loss: {train_loss:.4f}")

        # Validate
        val_loss = eval_fn(model, test_loader, Config.DEVICE)
        print(f"   Val Loss: {val_loss:.4f}")

        # Update learning rate
        scheduler.step(val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        print(f"   Learning Rate: {current_lr:.6f}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_captcha_model.pth')
            print(f"   ✓ Best model saved (Val Loss: {val_loss:.4f})")

    # ========== STEP 6: FINAL EVALUATION ==========
    print("\n[6/6] Training complete!")
    print("="*70)
    print(f"Best Validation Loss: {best_val_loss:.4f}")
    print("\nModel saved as: best_captcha_model.pth")

    return model, lbl_enc

In [7]:
model, label_encoder = run_training()

CAPTCHA RECOGNITION TRAINING

[1/6] Loading image paths...
   Train images: 79016
   Val images: 19755

[2/6] Encoding labels...
   Unique characters: 60
   Characters: [np.str_('1'), np.str_('2'), np.str_('3'), np.str_('4'), np.str_('5'), np.str_('6'), np.str_('7'), np.str_('8'), np.str_('9'), np.str_('A'), np.str_('B'), np.str_('C'), np.str_('D'), np.str_('E'), np.str_('F'), np.str_('G'), np.str_('H'), np.str_('I'), np.str_('J'), np.str_('K'), np.str_('L'), np.str_('M'), np.str_('N'), np.str_('O'), np.str_('P'), np.str_('Q'), np.str_('R'), np.str_('S'), np.str_('T'), np.str_('U'), np.str_('V'), np.str_('W'), np.str_('X'), np.str_('Y'), np.str_('Z'), np.str_('a'), np.str_('b'), np.str_('c'), np.str_('d'), np.str_('e'), np.str_('f'), np.str_('g'), np.str_('h'), np.str_('i'), np.str_('j'), np.str_('k'), np.str_('l'), np.str_('m'), np.str_('n'), np.str_('p'), np.str_('q'), np.str_('r'), np.str_('s'), np.str_('t'), np.str_('u'), np.str_('v'), np.str_('w'), np.str_('x'), np.str_('y'), np.s

Training: 100%|██████████| 2470/2470 [05:01<00:00,  8.20it/s]


   Train Loss: 4.4141


Validation: 100%|██████████| 618/618 [01:09<00:00,  8.90it/s]


   Val Loss: 4.2237
   Learning Rate: 0.001000
   ✓ Best model saved (Val Loss: 4.2237)

Epoch 2/150
----------------------------------------------------------------------


...

   Val Loss: 0.8522
   Learning Rate: 0.000250
   ✓ Best model saved (Val Loss: 0.8522)

Epoch 150/150
----------------------------------------------------------------------


Training: 100%|██████████| 2470/2470 [00:48<00:00, 51.29it/s]


   Train Loss: 0.9513


Validation: 100%|██████████| 618/618 [00:11<00:00, 51.96it/s]


   Val Loss: 0.8517
   Learning Rate: 0.000250
   ✓ Best model saved (Val Loss: 0.8517)

[6/6] Training complete!
Best Validation Loss: 0.8517

Model saved as: best_captcha_model.pth


In [8]:
def remove_duplicates(x):
    """Remove consecutive duplicate characters"""
    if len(x) < 2:
        return x
    fin = ""
    for j in x:
        if fin == "":
            fin = j
        else:
            if j == fin[-1]:
                continue
            else:
                fin = fin + j
    return fin


def decode_predictions(preds, encoder):
    
    preds = preds.permute(1, 0, 2)  # batch, sequence_length, num_classes
    preds = torch.softmax(preds, 2)
    preds = torch.argmax(preds, 2)
    preds = preds.detach().cpu().numpy()

    cap_preds = []
    for j in range(preds.shape[0]):
        temp = []
        for k in preds[j, :]:
            k = k - 1
            if k == -1:
                temp.append("§")  # Blank token placeholder
            else:
                p = encoder.inverse_transform([k])[0]
                temp.append(p)
        tp = "".join(temp).replace("§", "")
        cap_preds.append(remove_duplicates(tp))

    return cap_preds

In [9]:

def predict_single_image(image_path, model, lbl_enc, device='cpu'):

    # Use ClassificationDataset for preprocessing (same as training)
    dataset = ClassificationDataset(
        image_paths=[image_path],
        labels=[[0, 0, 0, 0, 0]]  # Dummy labels, not used for prediction
    )

    # Get preprocessed image
    image_dict = dataset[0]
    image_tensor = image_dict['images'].unsqueeze(
        0).to(device)  # Add batch dimension

    # Predict
    with torch.no_grad():
        preds, _ = model(image_tensor)

    # Decode
    text = decode_predictions(preds, lbl_enc)[0]

    return text

In [10]:
test_image = "/kaggle/input/captcha-dataset-98k/Dataset/val/8VKEx.jpg"  # Change to your image path
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if os.path.exists(test_image):
    predicted_text = predict_single_image(test_image, model, label_encoder, DEVICE)
    actual_text = test_image.split(os.sep)[-1].split('.')[0]

    print(f"\nImage: {test_image}")
    print(f"Predicted: {predicted_text}")
    print(f"Actual: {actual_text}")
    print(f"Match: {'✓' if predicted_text == actual_text else '✗'}")
else:
    print(f"Test image not found: {test_image}")


Image: /kaggle/input/captcha-dataset-98k/Dataset/val/8VKEx.jpg
Predicted: 8VKEx
Actual: 8VKEx
Match: ✓


In [11]:
import pickle
def save_label_encoder(lbl_enc, save_path='label_encoder.pkl'):
    """Save LabelEncoder for later use"""
    with open(save_path, 'wb') as f:
        pickle.dump(lbl_enc, f)
    print(f"Label encoder saved to {save_path}")

save_label_encoder(label_encoder)

Label encoder saved to label_encoder.pkl
